In [ ]:
!pip install earthengine-api


In [ ]:
!pip install geopandas

In [26]:
import ee
import geopandas as gpd
import json

# Authentification et initialisation
ee.Authenticate()
ee.Initialize(project="ee-syllamina")


In [28]:
# Chemin vers le GeoJSON
geojson_path = r"C:\Users\admin\Desktop\Tolbi 2025\Cluster_Sénégal_31X31\output_20_clusters_senegal\cluster_100.geojson"

# Lire le GeoJSON avec geopandas
gdf = gpd.read_file(geojson_path)

# Convertir en dictionnaire GeoJSON
geojson_dict = json.loads(gdf.to_json())

# Convertir en FeatureCollection Earth Engine
poly_fc = ee.FeatureCollection(geojson_dict)

# Extraire la géométrie
poly = poly_fc.geometry()

print("Polygone chargé avec succès")


Polygone chargé avec succès


2. SOILGRIDS – TEXTURE

In [29]:
sand = ee.Image('projects/soilgrids-isric/sand_mean').select('sand_0-5cm_mean')
silt = ee.Image('projects/soilgrids-isric/silt_mean').select('silt_0-5cm_mean')
clay = ee.Image('projects/soilgrids-isric/clay_mean').select('clay_0-5cm_mean')
soc  = ee.Image('projects/soilgrids-isric/soc_mean').select('soc_0-5cm_mean')

stats = ee.Image.cat([sand, silt, clay, soc]).reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=poly,
    scale=250,
    maxPixels=1e13
)


3. EXTRACTION ZONALE

In [30]:
stats = ee.Image.cat([sand, silt, clay, soc]).reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=poly,
    scale=250,
    maxPixels=1e13
)

sandPct = ee.Number(stats.get('sand_0-5cm_mean'))
siltPct = ee.Number(stats.get('silt_0-5cm_mean'))
clayPct = ee.Number(stats.get('clay_0-5cm_mean'))
socPct  = ee.Number(stats.get('soc_0-5cm_mean'))

print("Pourcentages (%) :")
print({
    "sand": sandPct.getInfo(),
    "silt": siltPct.getInfo(),
    "clay": clayPct.getInfo(),
    "SOC": socPct.getInfo()
})


Pourcentages (%) :
{'sand': 687.2030345164662, 'silt': 148.19545350851558, 'clay': 164.5980887222581, 'SOC': 150.89955633981126}


4. CLASSE TEXTURALE USDA

In [34]:
aquacropSoil = ee.Algorithms.If(
    sandPct.gte(85).And(clayPct.lt(10)), 'Sand',
    ee.Algorithms.If(
        sandPct.gte(70).And(clayPct.lt(15)), 'LoamySand',
        ee.Algorithms.If(
            sandPct.gte(43).And(sandPct.lt(85))
            .And(clayPct.gte(7)).And(clayPct.lt(20)), 'SandyLoam',
            ee.Algorithms.If(
                siltPct.gte(80).And(clayPct.lt(12)), 'Silt',
                ee.Algorithms.If(
                    siltPct.gte(50).And(clayPct.lt(27)), 'SiltLoam',
                    ee.Algorithms.If(
                        sandPct.gte(45).And(clayPct.gte(35)), 'SandyClay',
                        ee.Algorithms.If(
                            siltPct.gte(40).And(clayPct.gte(40)), 'SiltyClay',
                            ee.Algorithms.If(
                                clayPct.gte(40), 'Clay',
                                ee.Algorithms.If(
                                    sandPct.gte(45).And(clayPct.gte(20)).And(clayPct.lt(35)),
                                    'SandyClayLoam',
                                    ee.Algorithms.If(
                                        siltPct.gte(40).And(clayPct.gte(27)).And(clayPct.lt(40)),
                                        'SiltyClayLoam',
                                        ee.Algorithms.If(
                                            clayPct.gte(27).And(clayPct.lt(40)),
                                            'ClayLoam',
                                            'Loam'
                                        )
                                    )
                                )
                            )
                        )
                    )
                )
            )
        )
    )
)

print("Texture AquaCrop :", ee.String(aquacropSoil).getInfo())


Texture AquaCrop : SandyClay


In [ ]:
import ee
import geopandas as gpd
import json
import pandas as pd

# -----------------------
# 1. Initialisation GEE
# -----------------------
ee.Authenticate()
ee.Initialize(project="ee-syllamina")

# -----------------------
# 2. Charger les polygones
# -----------------------
geojson_path = r"C:\Users\admin\Desktop\Tolbi 2025\Cluster_Sénégal_31X31\soil_fao_others_sections.geojson"
gdf = gpd.read_file(geojson_path)

# Ajouter un ID si absent
if 'fid' not in gdf.columns:
    gdf['fid'] = range(len(gdf))

# --- Convertir les colonnes datetime en string ---
for col in gdf.select_dtypes(include=['datetime64[ns]']).columns:
    gdf[col] = gdf[col].astype(str)

# --- Optionnel : garder seulement les colonnes nécessaires pour EE ---
cols_to_keep = ['fid', 'geometry']
gdf_clean = gdf[cols_to_keep]

# Convertir en FeatureCollection EE
fc = ee.FeatureCollection(json.loads(gdf_clean.to_json()))

# -----------------------
# 3. Charger SoilGrids (0–5 cm)
# -----------------------
sand = ee.Image('projects/soilgrids-isric/sand_mean').select('sand_0-5cm_mean')
silt = ee.Image('projects/soilgrids-isric/silt_mean').select('silt_0-5cm_mean')
clay = ee.Image('projects/soilgrids-isric/clay_mean').select('clay_0-5cm_mean')
soc  = ee.Image('projects/soilgrids-isric/soc_mean').select('soc_0-5cm_mean')

# -----------------------
# 4. Fonction de calcul pour chaque polygone
# -----------------------
def compute_soil(feature):
    geom = feature.geometry()
    
    stats = ee.Image.cat([sand, silt, clay, soc]).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geom,
        scale=250,
        maxPixels=1e13
    )
    
    sandPct = ee.Number(ee.Algorithms.If(stats.get('sand_0-5cm_mean'), stats.get('sand_0-5cm_mean'), 0))
    siltPct = ee.Number(ee.Algorithms.If(stats.get('silt_0-5cm_mean'), stats.get('silt_0-5cm_mean'), 0))
    clayPct = ee.Number(ee.Algorithms.If(stats.get('clay_0-5cm_mean'), stats.get('clay_0-5cm_mean'), 0))
    socPct  = ee.Number(ee.Algorithms.If(stats.get('soc_0-5cm_mean'),  stats.get('soc_0-5cm_mean'),  0))

    # Normalisation %
    total = sandPct.add(siltPct).add(clayPct)
    total = ee.Algorithms.If(total.eq(0), 1, total)
    total = ee.Number(total)

    sandPct = sandPct.divide(total).multiply(100)
    siltPct = siltPct.divide(total).multiply(100)
    clayPct = clayPct.divide(total).multiply(100)

    # Classe USDA
    # Classe USDA avec intervalles ajustés pour correspondance exacte AquaCrop
    usda = ee.Algorithms.If(
    sandPct.gte(85).And(clayPct.lt(10)), 'Sand',                     # très sableux
    ee.Algorithms.If(
        sandPct.gte(70).And(clayPct.lt(15)), 'Loamy Sand',           # LoamySand en AquaCrop
        ee.Algorithms.If(
            sandPct.gte(43).And(sandPct.lt(70)).And(clayPct.gte(7)).And(clayPct.lt(20)), 'Sandy Loam',
            ee.Algorithms.If(
                siltPct.gte(80).And(clayPct.lt(12)), 'Silt',
                ee.Algorithms.If(
                    siltPct.gte(50).And(clayPct.lt(27)), 'Silt Loam',
                    ee.Algorithms.If(
                        sandPct.gte(45).And(clayPct.gte(35)), 'Sandy Clay',
                        ee.Algorithms.If(
                            siltPct.gte(40).And(clayPct.gte(40)), 'Silty Clay',
                            ee.Algorithms.If(
                                clayPct.gte(40), 'Clay',
                                ee.Algorithms.If(
                                    sandPct.gte(45).And(clayPct.gte(20)).And(clayPct.lt(35)), 'Sandy Clay Loam',
                                    ee.Algorithms.If(
                                        siltPct.gte(40).And(clayPct.gte(27)).And(clayPct.lt(40)), 'Silty Clay Loam',
                                        ee.Algorithms.If(
                                            clayPct.gte(27).And(clayPct.lt(40)), 'Clay Loam',
                                            'Loam'  # cas général
                                        ))
                                    ))
                                ))
                            ))
                        ))
                    )
            



    # Conversion USDA → AquaCrop
    aquacrop_texture = ee.Algorithms.If(
        ee.String(usda).equals('Sand'), 'Sand',
        ee.Algorithms.If(
            ee.String(usda).equals('Loamy Sand'), 'LoamySand',
            ee.Algorithms.If(
                ee.String(usda).equals('Sandy Loam'), 'SandyLoam',
                ee.Algorithms.If(
                    ee.String(usda).equals('Loam'), 'Loam',
                    ee.Algorithms.If(
                        ee.String(usda).equals('Silt Loam'), 'SiltLoam',
                        ee.Algorithms.If(
                            ee.String(usda).equals('Silt'), 'Silt',
                            ee.Algorithms.If(
                                ee.String(usda).equals('Sandy Clay Loam'), 'SandyClayLoam',
                                ee.Algorithms.If(
                                    ee.String(usda).equals('Clay Loam'), 'ClayLoam',
                                    ee.Algorithms.If(
                                        ee.String(usda).equals('Silty Clay Loam'), 'SiltClayLoam',
                                        ee.Algorithms.If(
                                            ee.String(usda).equals('Sandy Clay'), 'SandyClay',
                                            ee.Algorithms.If(
                                                ee.String(usda).equals('Silty Clay'), 'SiltyClay',
                                                'Clay'
                                            )
                                        )
                                    )
                                )
                            )
                        )
                    )
                )
            )
        )
    )

    return feature.set({
        'fid': feature.get('fid'),
        'sand_pct': sandPct,
        'silt_pct': siltPct,
        'clay_pct': clayPct,
        'soc': socPct,
        'aquacrop_texture': aquacrop_texture
    })


# -----------------------
# 5. Appliquer à tous les polygones
# -----------------------
results_fc = fc.map(compute_soil)

# -----------------------
# 6. Récupération côté Python
# -----------------------
features = results_fc.getInfo()['features']

rows = []
for f in features:
    p = f['properties']
    rows.append({
        'fid': p['fid'],
        'sand_pct': p['sand_pct'],
        'silt_pct': p['silt_pct'],
        'clay_pct': p['clay_pct'],
        'soc': p['soc'],
        'aquacrop_texture': p['aquacrop_texture']
    })

df = pd.DataFrame(rows)

# -----------------------
# 7. Sauvegarde GeoJSON
# -----------------------
# Il faut reconstruire un GeoDataFrame pour GeoJSON
gdf_out = gdf_clean.merge(df, on='fid')
gdf_out.to_file(r"C:\Users\admin\Desktop\soil_aquacrop_others_sections.geojson", driver='GeoJSON')

print("✅ Résultats sauvegardés en GeoJSON")
print(gdf_out.head())
